# Cartesian Compliance

## Import Modules

In [ ]:
import numpy as np

import matplotlib as mpl
mpl.use('Qt5Agg')

import matplotlib.pyplot as plt
plt.ion()

# for the symbolic manipulation of jacobian
import sympy as sp
from sympy.utilities.lambdify import lambdify

from scipy.optimize import minimize
from scipy.optimize import fsolve

import time

import odrive
import odrive.utils
import odrive.enums

## Define leg assembly related variables

In [ ]:
# code shared by professor
## Motor constants
K = 0.0285;                 # Nm / A
peak_amp = 30;              # A
peak_torque = K * peak_amp; # Nm
m_motor = 0.2;              # kg
m_assembly = 0.2;           # kg

m_total = 2*m_motor + m_assembly; # kg

gravity = 9.8;              # m/s^2

weight = m_total * gravity  # N

l1 = 0.09;                  # m 
l2 = 0.16;                  # m
w = 0.07;                   # m

# solve jacobian of constraint equation
(thetaL_sym, 
 thetaR_sym, 
 link1_sym, 
 link2_sym, 
 width_sym) = sp.symbols("""thetaL_sym 
                            thetaR_sym 
                            link1_sym 
                            link2_sym 
                            width_sym""", real = True)

def T(theta, x, y):
    """
    Function to return an arbitrary transformation matrix 
    This is for sympy symbolic calculation
    """
    return sp.Matrix([[sp.cos(theta), -sp.sin(theta), x], 
                      [sp.sin(theta), sp.cos(theta), y],
                      [0, 0, 1]])

def sym_to_np(T):
    return np.array(T).astype(np.float64)

## Solve the FK

In [ ]:
x_r = width_sym/2 + link1_sym*sp.cos(thetaR_sym)
x_l = -width_sym/2 + link1_sym*sp.cos(thetaL_sym)

y_r = link1_sym*sp.sin(thetaR_sym)
y_l = link1_sym*sp.sin(thetaL_sym)

theta3_sym = sp.atan2(y_r - y_l, x_r - x_l)
L = sp.sqrt((x_l - x_r)**2 + (y_l - y_r)**2)

FK = T(thetaL_sym, -width_sym/2, 0)@T(-(thetaL_sym - theta3_sym), link1_sym, 0)@sp.Matrix([L/2, sp.sqrt(link2_sym**2 - (L/2)**2), 1])
FK = FK[:2,:]
FK.simplify()
FK

## Find jacobian to find time-derivate of cartesian coordinates 

In [ ]:
xy = FK.subs([(link1_sym, l1), (link2_sym, l2), (width_sym, w)]).evalf()
J = xy.jacobian([thetaL_sym, thetaR_sym]).evalf()
J = lambdify((thetaL_sym, thetaR_sym), J)

## Define helper functions

In [ ]:
def findCoordOfEndPt(thetaL, thetaR):
    M = FK.subs([(thetaL_sym, thetaL),(thetaR_sym, thetaR),(link1_sym, l1),(link2_sym, l2),(width_sym, w)])
    x, y = sym_to_np(M).reshape((1,2))[0]
    return x, y

def getAngleL(count, zerocount, motor_cpr):
    thetaL = 2*np.pi*(1/2-(count - zerocount)/motor_cpr)
    return thetaL

def getAngleR(count, zerocount, motor_cpr):
    thetaR = -2*np.pi*(count - zerocount)/motor_cpr
    return thetaR

def rad2deg(angle):
    return angle * (180/np.pi)

## Connect to ODrive and Calibrate

In [ ]:
# CONNECT TO ODRIVE

odrv0 = odrive.find_any()
if odrv0 is not None:
    print('Connected!')
    print('Odrive serial {}'.format(odrv0.serial_number))
    
    m0 = odrv0.axis0.motor.is_calibrated
    m1 = odrv0.axis1.motor.is_calibrated
    
    print('Motor 0 calibrated: {}'.format(m0))
    print('Motor 1 calibrated: {}'.format(m1))    
    
else:
    print('Not connected')
    

In [ ]:
# CALIBRATE

odrv0.axis0.requested_state = odrive.enums.AXIS_STATE_FULL_CALIBRATION_SEQUENCE
odrv0.axis1.requested_state = odrive.enums.AXIS_STATE_FULL_CALIBRATION_SEQUENCE

time.sleep(15)

print('\t Motor 0 calibration result: {} \r\n'.format(odrv0.axis0.motor.is_calibrated), 
      '\t Motor 1 calibration result: {}'.format(odrv0.axis1.motor.is_calibrated))


## Define zero pos estimate (useful for calculating ThetaL and ThetaR)

In [ ]:
motor_cpr = (odrv0.axis0.encoder.config.cpr, 
             odrv0.axis1.encoder.config.cpr)

zero_L = odrv0.axis0.encoder.pos_estimate
zero_R = odrv0.axis1.encoder.pos_estimate
print(zero_L, zero_R)

## Define cartesian spring's equilibrium position

In [ ]:
eqCountL = odrv0.axis0.encoder.pos_estimate
eqCountR = odrv0.axis1.encoder.pos_estimate

print('countL is {}, countR is {}'.format(eqCountL, eqCountR))

eqThetaL = getAngleL(eqCountL, zero_L, motor_cpr[0])
eqThetaR = getAngleR(eqCountR, zero_R, motor_cpr[1])

print('eqThetaL is {}, eqThetaR is {}'.format(eqThetaL, eqThetaR))

M = FK.subs([(thetaL_sym, eqThetaL),(thetaR_sym, eqThetaR),(link1_sym, l1),(link2_sym, l2),(width_sym, w)])
eqX, eqY = sym_to_np(M).reshape((1,2))[0]

## Clear any errors and put motors in closed loop control mode

In [ ]:
odrv0.axis0.controller.error=0
odrv0.axis0.motor.error=0
odrv0.axis0.encoder.error=0
odrv0.axis0.error=0

odrv0.axis1.controller.error=0
odrv0.axis1.motor.error=0
odrv0.axis1.encoder.error=0
odrv0.axis1.error=0

# current limit
curr_limit = 25 #A
odrv0.axis0.motor.config.current_lim = curr_limit
odrv0.axis1.motor.config.current_lim = curr_limit

# set errors to 0
odrv0.axis0.controller.error=0
odrv0.axis0.motor.error=0
odrv0.axis0.encoder.error=0
odrv0.axis0.error=0

odrv0.axis1.controller.error=0
odrv0.axis1.motor.error=0
odrv0.axis1.encoder.error=0
odrv0.axis1.error=0

odrv0.axis0.requested_state = odrive.enums.AXIS_STATE_CLOSED_LOOP_CONTROL
odrv0.axis1.requested_state = odrive.enums.AXIS_STATE_CLOSED_LOOP_CONTROL


In [ ]:
odrv0.axis0.controller.set_pos_setpoint(eqCountL,0,0)
odrv0.axis1.controller.set_pos_setpoint(eqCountR,0,0)

## Make leg assembly behave as polar spring

### Approach

I use the jacobian matrix J and angular velocity of the motors to find the time rate change of the x and y position of the foot (ie xdot, and ydot).

For damping behavior I use `xdot` and `ydot` (ie time rate change of the x and y position) to apply forces that resist velocity for both x direction and y direction.

I also damp the motors individually by a term propotional to thetaLDot and thetaRDot respectively for the left and right motors. The gain for this damping is `Ctheta` (see below).

In order to find the current that has to be sent to the motors, I simply add the current calculated from the individual motor damping and the cartesian spring model calculation of currents using the force in x and y directions.

In [ ]:
# Kx=30 #N/m 25 working with 15 amp limit
Kx=30 #N/m

# Ky=60 #N/m 60 working with 15 amp limit
Ky=0

Cx=0 #N sec
Cy=0 #N sec

curr_limit = 25 # Amp

# -Ctheta*thetaDot
Ctheta = 0.8 # Nm sec / rad

from time import sleep

while True:
    countL = odrv0.axis0.encoder.pos_estimate
    countR = odrv0.axis1.encoder.pos_estimate
    
    actualThetaL = getAngleL(countL,zero_L, motor_cpr[0])
    actualThetaR = getAngleR(countR,zero_R, motor_cpr[1])
    
    x, y = findCoordOfEndPt(actualThetaL, actualThetaR)
    
    delX = x - eqX
    delY = y - eqY
    
    Jval = J(actualThetaL, actualThetaR)
    
    # postive clockwise
    thetaLDot = (odrv0.axis0.encoder.vel_estimate * (2*np.pi)) / motor_cpr[0]
    thetaRDot = (odrv0.axis1.encoder.vel_estimate * (2*np.pi)) / motor_cpr[1]
    
    angVel = np.array([[thetaLDot],[thetaRDot]])
    
    xdot = (Jval @ angVel)[0][0]
    ydot = (Jval @ angVel)[1][0]

    forceX = - Kx*delX - Cx*xdot
    forceY = - Ky*delY - Cy*ydot
        
    force = np.array([[forceX],[forceY]])
        
    current = (1/K) * Jval.T @ force
    
    currentL = -current[0]
    currentR = -current[1]
    
    currL = -Ctheta*thetaLDot + currentL
    currR = -Ctheta*thetaRDot + currentR
    
    currL = min(np.abs(currL), curr_limit) * np.sign(currL)
    currR = min(np.abs(currR), curr_limit) * np.sign(currR)
    
    odrv0.axis0.controller.set_current_setpoint(currL)
    odrv0.axis1.controller.set_current_setpoint(currR)

time.sleep(3)
odrv0.axis0.controller.set_pos_setpoint(zero_L,0,0)
odrv0.axis1.controller.set_pos_setpoint(zero_R,0,0)
time.sleep(3)
odrv0.axis0.requested_state = odrive.enums.AXIS_STATE_IDLE
odrv0.axis1.requested_state = odrive.enums.AXIS_STATE_IDLE

## Put ODrive in IDLE mode

In [ ]:
odrv0.axis0.requested_state = odrive.enums.AXIS_STATE_IDLE
odrv0.axis1.requested_state = odrive.enums.AXIS_STATE_IDLE

## Delete ODrive object

In [ ]:
del odrv0